In [4]:
import pandas as pd
import numpy as np

#Student number is 425288 so brand 88 (B&B, FLSMN, PKY, REST BRAND margarine tube market 3)
brand = "brand88"

data = pd.DataFrame({
    "logsales": pd.read_excel("sales.xls")[brand].apply(np.log),
    "logprice": pd.read_excel("price.xls")[brand].apply(np.log),
    "display": pd.read_excel("displ.xls")[brand],
    "coupon": pd.read_excel("coupon.xls")[brand]
})

data["intercept"] = 1

y =  data["logsales"]
X = data[["intercept", "logprice", "display", "coupon"]]

data.describe()

,logsales,logprice,display,coupon,intercept
count,124.000000,124.000000,124.000000,124.000000,124.0
mean,4.536251,0.296617,0.088710,0.854839,1.0
std,0.256217,0.016849,0.285478,0.353692,0.0
min,3.828641,0.195238,0.000000,0.000000,1.0
25%,4.356709,0.288781,0.000000,1.000000,1.0
50%,4.564348,0.297397,0.000000,1.000000,1.0
75%,4.718499,0.304982,0.000000,1.000000,1.0
max,5.257495,0.330023,1.000000,1.000000,1.0


Frequentist estimation

In [5]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression(fit_intercept=False).fit(X, y)

print("R2 ",reg.score(X, y))

print("\nCoefficients",reg.coef_)

R2  0.04064190974067994

Coefficients [ 4.12687887  1.57347144  0.14798799 -0.08244147]


Conditional distribution drawing

In [6]:
def draw_sigma(X,y,beta):
    #N + k
    df = len(y)+ len(beta)
    
    #(y-XB)
    e = y - np.dot(X,beta)
    
    #(y-XB)'(y-XB) + B'B
    alpha = np.dot(e.T,e) + np.dot(beta,beta)
    
    return alpha / np.random.chisquare(df)

def draw_beta(X,y,var):
    # X'X + I4
    inv = np.dot(X.T,X)+np.identity(4)
    # (X'X + I4)-1
    inv = np.linalg.inv(inv)
    
    # (X'X + I4)-1 X'y
    mean = np.dot(np.dot(inv, X.T) ,y)
    
    # σ2 (X'X + I4)-1
    cov = np.dot(var, inv)
    
    return np.random.multivariate_normal(mean, cov)

Gibbs Sampler

In [7]:
def gibbs_sampler(X,y,n):
    estimates =  np.zeros((n+1, 5))
    for i in range(n):
        estimates[i+1,4] = draw_sigma(X,y, estimates[i,0:4])
        estimates[i+1,0:4] = draw_beta(X,y,estimates[i+1,4])
        
    return pd.DataFrame(estimates, columns=['B0', 'B1', 'B2', 'B3', 'Sigma'])
    
samples = gibbs_sampler(X,y,500000)
display(samples.head())

,B0,B1,B2,B3,Sigma
0,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.680060,13.277332,0.298386,0.115553,26.980314
2,4.335553,0.642716,1.256672,-0.015263,1.646704
3,4.336727,0.604283,0.355509,-0.080944,0.297854
4,3.842626,1.697405,0.138396,0.127687,0.255147


Estimates

In [8]:
burnin = 50000
cleaned = samples.iloc[burnin:,:].agg(["mean", "var"]).transpose()

#no thin value because it makes the estimates less precise, and plotting or limited space are not of interest

print("Estimates of the full sample\n")
original = samples.agg(["mean", "var"]).transpose()
display(original)

print("\nEstimates cleaned of burnin of ",burnin," samples\n")
display(cleaned)

print("\nDifference between full sample and cleaned sample\n")

display(original-cleaned)

Estimates of the full sample



,mean,var
B0,4.017838,0.026553
B1,1.219143,0.192657
B2,0.131619,0.019840
B3,0.131848,0.012821
Sigma,0.215295,0.002210



Estimates cleaned of burnin of  50000  samples



,mean,var
B0,4.017832,0.026492
B1,1.219145,0.192359
B2,0.131641,0.019823
B3,0.131857,0.012812
Sigma,0.215257,0.000773



Difference between full sample and cleaned sample



,mean,var
B0,0.000006,0.000061
B1,-0.000002,0.000298
B2,-0.000022,0.000017
B3,-0.000009,0.000008
Sigma,0.000039,0.001437


Posterior odds

In [51]:
from scipy.stats import norm
# Get the mean and variance of posterior B1
mean, var = cleaned.iloc[1]

#P[B1 < 0 | y] = P[B1 / Stdev < 0 | y] = P[Z < 0]

prob = norm.cdf(0, loc=mean,scale=np.sqrt(var))

odds= (1 - prob)/prob

print("The posterior odds is ",odds)

The posterior odds is  366.5941512691721
